In [34]:
import sys

import sklearn
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
import numpy as np

from sklearn.model_selection import GridSearchCV

from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_val_score



from sklearn.pipeline import FeatureUnion

from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import Imputer

from sklearn.preprocessing import StandardScaler 
from sklearn.feature_extraction.text import TfidfVectorizer

%matplotlib inline



train = pd.read_csv('data/titanic/train.csv', index_col=['PassengerId'])
test = pd.read_csv('data/titanic/test.csv', index_col=['PassengerId'])

In [35]:
test

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
897,3,"Svensson, Mr. Johan Cervin",male,14.0,0,0,7538,9.2250,NaN,S
898,3,"Connolly, Miss. Kate",female,30.0,0,0,330972,7.6292,NaN,Q
899,2,"Caldwell, Mr. Albert Francis",male,26.0,1,1,248738,29.0000,NaN,S
900,3,"Abrahim, Mrs. Joseph (Sophie Halaut Easu)",female,18.0,0,0,2657,7.2292,NaN,C


## разделям тренировачния дата сет


In [36]:
from sklearn.model_selection import train_test_split
train_no_survived = train.drop(labels=['Survived'], axis=1)
train_suvived = train['Survived']



x_train, x_test, y_train, y_test = train_test_split(train_no_survived, 
                                                    train_suvived, 
                                                    test_size=0.3, 
                                                    random_state=4330)

In [37]:
numeric_column_names = x_train.describe().columns
train_no_survived.Age.fillna(0)
train = train_no_survived.fillna("")

In [38]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 623 entries, 495 to 57
Data columns (total 10 columns):
Pclass      623 non-null int64
Name        623 non-null object
Sex         623 non-null object
Age         502 non-null float64
SibSp       623 non-null int64
Parch       623 non-null int64
Ticket      623 non-null object
Fare        623 non-null float64
Cabin       135 non-null object
Embarked    621 non-null object
dtypes: float64(2), int64(3), object(5)
memory usage: 53.5+ KB


In [39]:

x_train.Embarked.value_counts()

S    444
C    119
Q     58
Name: Embarked, dtype: int64

### създавам трансформатор който енкодва колона и я връща присиединена с оригиналните колони


In [40]:
from sklearn.base import TransformerMixin
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

def encode(categorical_data):
    encoders = {col: LabelEncoder().fit(train[col]) for col in categorical_data.columns}
    categorical_data = categorical_data.fillna("")
    train_encoded = pd.DataFrame({col: encoders[col].transform(categorical_data[col]) for col in categorical_data.columns},
                        index = categorical_data.index)
    oh = OneHotEncoder().fit(train_encoded)
    return oh.transform(train_encoded)
 

class Encode_categorical(TransformerMixin): 
    def __init__(self, *column_names):
        self.column_names = column_names
        
    def fit(self, *_):
        return self

    def transform(self, X, *_):
        #removes columns and replaces them with 
        
        column_names = self.column_names[0]
        
        encoded_df = encode(X[column_names]).todense()
        
        X_clean_columns = X.columns.drop(column_names)
        print(X[X_clean_columns])
    
        print(X_clean_columns)
        return X[X_clean_columns].join(pd.DataFrame(encoded_df))

In [41]:
from sklearn.base import BaseEstimator, TransformerMixin
class ItemSelector(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key

    def fit(self, x, y=None):
        return self

    def transform(self, data_dict):
        return data_dict[[self.key]]

        създаваме селектор който да улесни работата 

In [42]:
class BinarizerPipe(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None,**fit_params):
        return self
    def transform(self, X):
        return LabelBinarizer().fit(X).transform(X)

        създавам мои си трансформатор защото  LabelBinarizer - а не има проблеми с pipeline - ове

In [43]:


union = FeatureUnion([
    ('age', Pipeline([
        ('select', ItemSelector('Age')),
        ('imputer', Imputer(strategy='mean')),
        ('scaler', StandardScaler()),
    ])),
    ('name', Pipeline([
            ('select', ItemSelector('Name')),            
            ('vectorize', CountVectorizer()),
    ]))
    
    
])
 

In [44]:
model = Pipeline([
    ('union', union),
    ('svc', SVC())
])

# Всъщност нямаше нужда от горните глупости и дефиниции ама ме яд да ги изтрия  :Д

## Ше използвам LinearSVC и feature-а Name, който попринцип игнорирахме или box-вах ме.



In [45]:
train = pd.read_csv('data/Titanic/train.csv', index_col=['PassengerId'])
test = pd.read_csv('data/Titanic/test.csv', index_col=['PassengerId'])

from sklearn.model_selection import train_test_split
train_no_survived = train.drop(labels=['Survived'], axis=1)
train_suvived = train['Survived']




pipeline = Pipeline([
    ('features', CountVectorizer()),
    ('clf', LinearSVC())
])
pipeline.fit(x_train["Name"],y_train)
print(pipeline.score(x_train["Name"],y_train))
print(pipeline.score(x_test["Name"],y_test))

1.0
0.80223880597


In [46]:
scores = cross_val_score(pipeline, x_test['Name'], y_test)
print(scores)
print(scores.mean())

[ 0.83333333  0.73333333  0.84090909]
0.802525252525


Изглежда добре сега ще тренираме с целия train sample

In [47]:
pipeline.fit(train_no_survived["Name"],train_suvived)

Pipeline(steps=[('features', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        s...ax_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))])

In [48]:
pred = pipeline.predict(test['Name'])


In [49]:
frame = pd.DataFrame({
    'PassengerId': pd.read_csv('data/titanic/test.csv').PassengerId,
    'Survived': pred
})
frame = frame.set_index('PassengerId')
frame.to_csv('predictionsfine.csv')
frame.head()

,Survived
PassengerId,
892,0
893,1
894,0
895,0
896,1


![img na score](myscore.png "Резултат")

## Което е подобрение от 0.7894 (job's done)